In [2]:
!pip install transformers
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from transformers import BertTokenizer, BertForSequenceClassification
from prettytable import PrettyTable
import torch
from json.decoder import JSONDecodeError

# Get a list of tickers from Wikipedia's list of S&P 500 companies
# (rest of the code remains the same)

# Fetch news headlines for selected ticker using yfinance
def get_ticker_news(ticker):
    ticker_data = yf.Ticker(ticker)
    try:
        return ticker_data.news
    except JSONDecodeError:
        print(f"Failed to fetch news for ticker {ticker}.")
        return []

def get_ticker_news(ticker):
    ticker_data = yf.Ticker(ticker)
    try:
        return ticker_data.news
    except JSONDecodeError:
        print(f"Failed to fetch news for ticker {ticker}.")
        return []

# Load the pretrained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = BertForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Function to predict sentiment
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment = torch.argmax(probs)
    labels = ['very negative', 'negative', 'neutral', 'positive', 'very positive']
    return labels[sentiment]
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.strip()
        tickers.append(ticker)
    return tickers

tickers = get_sp500_tickers()



# Get the list of available tickers
# (rest of the code remains the same)

print("Available Tickers:")
print(', '.join(tickers[:50]))

# Prompt the user to select a ticker
selected_ticker = input("\nEnter a ticker from the above or type 'ALL' to view all: ").upper()

# Initialize table
table = PrettyTable()
table.field_names = ["Ticker", "Title", "Sentiment", "Stock Name", "Link"]

# Mapping of tickers to stock names
ticker_to_stock_name = {
    "MMM": "3M",
    "AOS": "A. O. Smith Corporation",
    "ABT": "Abbott Laboratories",
    "ABBV": "AbbVie",
    "ACN": "Accenture",
    "ATVI": "Activision Blizzard",
    "ADM": "Archer-Daniels-Midland Company",
    "ADBE": "Adobe",
    "ADP": "Automatic Data Processing",
    "AAP": "Advance Auto Parts",
    "AES": "AES Corporation",
    "AFL": "Aflac",
    "A": "Agilent Technologies",
    "APD": "Air Products and Chemicals",
    "AKAM": "Akamai Technologies",
    "ALK": "Alaska Air Group",
    "ALB": "Albemarle Corporation",
    "ARE": "Alexandria Real Estate Equities",
    "ALGN": "Align Technology",
    "ALLE": "Allegion",
    "LNT": "Alliant Energy",
    "ALL": "Allstate",
    "GOOGL": "Alphabet Inc. (Class A)",
    "GOOG": "Alphabet Inc. (Class C)",
    "MO": "Altria Group",
    "AMZN": "Amazon",
    "AMCR": "Amcor plc",
    "AMD": "Advanced Micro Devices",
    "AEE": "Ameren",
    "AAL": "American Airlines Group",
    "AEP": "American Electric Power",
    "AXP": "American Express",
    "AIG": "American International Group",
    "AMT": "American Tower Corporation",
    "AWK": "American Water Works",
    "AMP": "Ameriprise Financial",
    "ABC": "AmerisourceBergen",
    "AME": "AMETEK",
    "AMGN": "Amgen",
    "APH": "Amphenol Corporation",
    "ADI": "Analog Devices",
    "ANSS": "ANSYS",
    "AON": "Aon",
    "APA": "Apache Corporation",
    "AAPL": "Apple",
    "AMAT": "Applied Materials",
    "APTV": "Aptiv PLC",
    "ACGL": "Arch Capital Group",
    "ANET": "Arista Networks",
    "AJG": "Arthur J. Gallagher & Co."
}


if selected_ticker != "ALL":
    news_list = get_ticker_news(selected_ticker)
    for news in news_list:
        title = news.get('title', 'N/A')
        if 'publishedDate' in news:
            timestamp = news['publishedDate']
        else:
            timestamp = 'N/A'
        sentiment = predict_sentiment(title)
        stock_name = ticker_to_stock_name.get(selected_ticker, "N/A")
        link = news.get('link', 'N/A')
        table.add_row([selected_ticker, title, sentiment, stock_name, link])
else:
    for ticker in tickers:
        news_list = get_ticker_news(ticker)
        for news in news_list:
            title = news.get('title', 'N/A')
            if 'publishedDate' in news:
                timestamp = news['publishedDate']
            else:
                timestamp = 'N/A'
            sentiment = predict_sentiment(title)
            stock_name = ticker_to_stock_name.get(ticker, "N/A")
            link = news.get('link', 'N/A')
            table.add_row([ticker, title, sentiment, stock_name, link])

print(table)


Available Tickers:
MMM, AOS, ABT, ABBV, ACN, ATVI, ADM, ADBE, ADP, AAP, AES, AFL, A, APD, AKAM, ALK, ALB, ARE, ALGN, ALLE, LNT, ALL, GOOGL, GOOG, MO, AMZN, AMCR, AMD, AEE, AAL, AEP, AXP, AIG, AMT, AWK, AMP, ABC, AME, AMGN, APH, ADI, ANSS, AON, APA, AAPL, AMAT, APTV, ACGL, ANET, AJG

Enter a ticker from the above or type 'ALL' to view all: ALL
+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+---------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------+
| Ticker |                                                                                                                                       Title                     